In [2]:
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [3]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/sebi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/sebi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sebi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/sebi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
medical_stopwords_keep = [
    "not", "no", "without", "never", "none", "neither", "nor", "n\'t"
    "always", "constant", "sudden", "recent", "persistent", "chronic", "acute",
    "left", "right", "upper", "lower", "side", "part", "area", "region", "zone",
    "feel", "seem", "have", "experience", "report", "complain",
    "mild", "severe", "moderate", "intense", "sharp", "dull", "worsening", "improving",
    "in", "on", "at", "during", "of"
]
medical_stopwords = [stopword for stopword in (stopwords.words('english') + ['\'s', ]) if stopword not in medical_stopwords_keep]

In [6]:
print('Medical stopwords:', medical_stopwords)
print(len(medical_stopwords))

Medical stopwords: ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'now', 'o', 'off', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she'd", "she'll", "she's", 'should', 's

In [7]:
import pandas as pd
df = pd.read_csv('symptom_description.csv')

In [8]:
df


,phrase1,phrase2,phrase3,phrase4,phrase5,phrase6,phrase7,phrase8,phrase9,phrase10,...,phrase22,phrase23,phrase24,phrase25,phrase26,phrase27,phrase28,phrase29,phrase30,symptom
0,My skin is itching constantly,"""I can’t stop scratching my arms""",There is a tingling sensation on my skin,"My back itches so much, it’s unbearable",I feel an intense urge to scratch my legs,My scalp is itching more than usual,This rash is making my skin extremely itchy,I woke up with an itchy sensation on my hands,My neck feels irritated and itchy,"There are red spots on my skin, and they itch ...",...,"My face is itchy, and it feels dry too",I scratched my arm so much that it turned red,My ears are itching inside,"My stomach itches, and I see tiny bumps",I can’t focus because my arms are itching so much,My lips feel itchy after eating that food,I have an intense itching sensation on my thighs,"My fingers are itchy, and I don’t know the reason",This new lotion is making my skin itch all over,itching
1,My skin has developed a red rash,There are small bumps appearing on my arms,I have an itchy rash spreading on my neck,My skin feels irritated and has turned red,A patch of rash appeared suddenly on my leg,The rash is warm to the touch,My hands are covered in red spots,I noticed a rough rash forming on my back,There are blotches of rash on my chest,My skin is peeling along with the rash,...,My back is covered in an itchy rash,The rash on my neck is turning darker,I feel a burning sensation along with the rash,The rash appeared suddenly after eating something,My skin is flaking where the rash is,The rash on my wrist is growing,My arms are full of tiny rash bumps,This rash is making my skin feel hot,I woke up with an unexplained rash on my thighs,skin_rash
2,My skin has developed raised nodules,There are hard lumps appearing on my arms,I have painful nodules forming on my skin,My skin feels swollen with small bumps,A cluster of nodules appeared suddenly on my leg,The nodules are firm and do not go away,My hands have multiple raised eruptions,I noticed tender nodules forming on my back,"There are small, swollen lumps on my chest",My skin is covered in thick nodules,...,My back is covered in thick skin eruptions,The lumps on my skin feel solid,The nodules on my neck are increasing,My skin is developing multiple firm nodules,These skin eruptions feel deep under the skin,The nodules are causing discomfort,My arms have several raised bumps,These nodules feel sore when touched,The eruptions are spreading across my shoulders,nodal_skin_eruptions
3,I keep sneezing repeatedly,My sneezing won't stop,I have been sneezing continuously since morning,Every few seconds I sneeze again,"My nose is irritated, and I can't stop sneezing",I sneeze non-stop whenever I step outside,Sneezing has been persistent all day,I can’t control my sneezing fits,I start sneezing continuously when I wake up,"Even without a cold, I keep sneezing",...,"I feel a tickle in my nose, followed by nonsto...","Once I start sneezing, I can't stop",I sneeze endlessly when I go outside,Strong perfumes make me sneeze repeatedly,Cold air makes me sneeze multiple times,Continuous sneezing is making my nose sore,I have sudden sneezing fits throughout the day,My sneezing happens in rapid succession,I can't stop sneezing whenever I enter an air-...,continuous_sneezing
4,I feel like I am constantly shivering,My body won’t stop shivering,I start shivering even when it's not cold,Shivering is making me feel weak,I can't control my shivering,My hands are shaking uncontrollably,I shiver even under a blanket,My whole body trembles suddenly,I get chills and start shivering,Shivering makes my teeth chatter,...,Shivering is making me feel exhausted,I suddenly start shaking uncontrollably,Shivering makes me feel dizzy,Even hot drinks don’t stop my shivering,I have chills that make me shiver,My whole body shakes from time to time,I shiver when I get nervous,I feel weak and keep shivering,Shivering won’t stop no matter w

In [9]:
def preprocess_description(description: str, wdl: WordNetLemmatizer, ps: PorterStemmer) -> str:
    description = description.lower()
    description_tokens = word_tokenize(description, language='english', preserve_line=True)
    description_tokens = [token for token in description_tokens if token not in (string.punctuation+"'\’`\'\'``") or token in medical_stopwords]
    words = [wdl.lemmatize(token) for token in description_tokens]
    words = list(filter(lambda word: word not in medical_stopwords, words))
    whitespaces = [' ' for _ in range(len(words))]
    return ''.join([word + whitespace for word, whitespace in zip(words, whitespaces)])

In [10]:
wdl = WordNetLemmatizer()
ps = PorterStemmer()

for column in df.columns[:-1]:
    for i in range(len(df[column])):
        df[column][i] = preprocess_description(df[column][i], wdl, ps)
df

/var/folders/mj/9j67468n7s90t742b29r6zr80000gn/T/ipykernel_75474/613525285.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[column][i] = preprocess_description(df[column][i], wdl, ps)


,phrase1,phrase2,phrase3,phrase4,phrase5,phrase6,phrase7,phrase8,phrase9,phrase10,...,phrase22,phrase23,phrase24,phrase25,phrase26,phrase27,phrase28,phrase29,phrase30,symptom
0,skin itching constantly,stop scratching arm,tingling sensation on skin,back itch much unbearable,feel intense urge scratch leg,scalp itching usual,rash making skin extremely itchy,woke itchy sensation on hand,neck feel irritated itchy,red spot on skin itch lot,...,face itchy feel dry,scratched arm much turned red,ear itching inside,stomach itch see tiny bump,focus arm itching much,lip feel itchy eating food,have intense itching sensation on thigh,finger itchy know reason,new lotion making skin itch,itching
1,skin ha developed red rash,small bump appearing on arm,have itchy rash spreading on neck,skin feel irritated ha turned red,patch of rash appeared suddenly on leg,rash warm touch,hand covered in red spot,noticed rough rash forming on back,blotch of rash on chest,skin peeling along rash,...,back covered in itchy rash,rash on neck turning darker,feel burning sensation along rash,rash appeared suddenly eating something,skin flaking rash,rash on wrist growing,arm full of tiny rash bump,rash making skin feel hot,woke unexplained rash on thigh,skin_rash
2,skin ha developed raised nodule,hard lump appearing on arm,have painful nodule forming on skin,skin feel swollen small bump,cluster of nodule appeared suddenly on leg,nodule firm not go away,hand have multiple raised eruption,noticed tender nodule forming on back,small swollen lump on chest,skin covered in thick nodule,...,back covered in thick skin eruption,lump on skin feel solid,nodule on neck increasing,skin developing multiple firm nodule,skin eruption feel deep skin,nodule causing discomfort,arm have several raised bump,nodule feel sore touched,eruption spreading across shoulder,nodal_skin_eruptions
3,keep sneezing repeatedly,sneezing wo n't stop,have sneezing continuously since morning,every second sneeze,nose irritated ca n't stop sneezing,sneeze non-stop whenever step outside,sneezing ha persistent day,control sneezing fit,start sneezing continuously wake,even without cold keep sneezing,...,feel tickle in nose followed nonstop sneezing,start sneezing ca n't stop,sneeze endlessly go outside,strong perfume make sneeze repeatedly,cold air make sneeze multiple time,continuous sneezing making nose sore,have sudden sneezing fit throughout day,sneezing happens in rapid succession,ca n't stop sneezing whenever enter air-condit...,continuous_sneezing
4,feel like constantly shivering,body stop shivering,start shivering even not cold,shivering making feel weak,ca n't control shivering,hand shaking uncontrollably,shiver even blanket,whole body tremble suddenly,get chill start shivering,shivering make teeth chatter,...,shivering making feel exhausted,suddenly start shaking uncontrollably,shivering make feel dizzy,even hot drink stop shivering,have chill make shiver,whole body shake time time,shiver get nervous,feel weak keep shivering,shivering stop no matter,shivering
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,have small dent in nail,small dent on fingernail,notice small indentation in nail,nail have tiny dent on,small dent in surface of nail,see small dent in nail,nail showing sign of small dent,noticed small dent appearing in nail,fingernail have little indentation,getting small dent in nail recently,...,nail have small mark dent,noticing tiny dent on nail,have small dent in nail,noticing little dent on nail,nail have tiny indentation,have dent in nail,have small dent in nail,noticing little dent on nail,nail have tiny indentation,small_dents_in_nails
128,nail appear inflamed,have inflamed nail,nail showing sign of inflammation,inflammation around nail,noticed inflammation on nail,nail swollen inflamed,area around nail look inflamed,have redness swelling around nail,noticeable inflammation on nail,nail painful inflamed,...,nail sore inflamed,dealing swollen 

In [11]:
from gensim.models import KeyedVectors

In [12]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [13]:
def get_tf_idf(sentences: list) -> dict:
    word_dict = {}
    word_document_occurence_dict = {}
    total_words = 0
    for sentence in sentences:
        set_words_in_document = set()
        for word in sentence.split():
            if word in model:
                total_words += 1
                if word in word_dict:
                    word_dict[word] += 1
                else:
                    word_dict[word] = 1
                set_words_in_document.add(word)
        for word in set_words_in_document:
            if word in word_document_occurence_dict:
                word_document_occurence_dict[word] += 1
            else:
                word_document_occurence_dict[word] = 1
    tf_dict = {word: (occurences / total_words) for word, occurences in word_dict.items()}
    idf_dict = {word: ((len(sentence) + 1) / (occurences + 1)) for word, occurences in word_document_occurence_dict.items()}
    tf_idf_dict = {word: (tf * idf_dict[word]) for word, tf in tf_dict.items()}
    return tf_idf_dict

In [14]:
def get_vectorized_symptom(sentences: list, model: KeyedVectors) -> np.ndarray:
    tf_idf_dict = get_tf_idf(sentences)
    symptom_vector = np.zeros(300)
    for sentence in sentences:
        sentence_vector = np.zeros(300)
        for word in sentence.split():
            if word in model:
                sentence_vector += model[word] * tf_idf_dict[word]
        symptom_vector += sentence_vector
    return symptom_vector / len(sentences)
    

In [15]:
vectorized_symptoms_dict = {row[len(row) - 1]: (get_vectorized_symptom(row[:-1].to_list(), model)) for _, row in df.iterrows()}
vectorized_symptoms_dict

/var/folders/mj/9j67468n7s90t742b29r6zr80000gn/T/ipykernel_75474/624149202.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vectorized_symptoms_dict = {row[len(row) - 1]: (get_vectorized_symptom(row[:-1].to_list(), model)) for _, row in df.iterrows()}


{'itching': array([ 4.93374229e-02,  4.73129754e-02, -2.31635600e-02,  1.30920408e-02,
        -1.35488633e-01,  6.92677498e-03, -4.13810471e-03, -1.28932387e-01,
         4.58366384e-03,  3.77074475e-02, -1.49952226e-02, -9.99123378e-02,
        -2.03102853e-02, -2.45294966e-02, -5.68175709e-02,  5.96570092e-02,
         4.05490868e-03,  1.49338183e-01,  2.30820058e-02, -8.29342502e-03,
        -6.26602162e-02,  4.51422632e-02,  5.66476839e-02, -2.17755640e-02,
        -6.38878334e-02, -7.79709218e-02,  4.17161981e-02,  4.26792818e-02,
         3.39711942e-02, -3.53846428e-02, -4.28062156e-02,  3.92526003e-02,
         2.43684694e-02, -2.63558312e-02, -5.10905172e-02, -4.16890716e-02,
         4.34835114e-02,  1.68785565e-02,  5.19907430e-02,  5.34413092e-02,
         6.52510287e-02, -1.08040746e-01,  6.54505771e-02, -1.18658880e-02,
         2.64064895e-02, -3.00707574e-02,  1.22003598e-02, -7.62393565e-03,
         2.11704511e-02,  2.44539059e-02, -1.11512046e-02,  9.54183028e-02,
 

In [16]:
cosine_similarity = lambda a, b: np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [17]:
cosine_similarity(vectorized_symptoms_dict['skin_rash'], vectorized_symptoms_dict['itching'])

0.8156006163052975

In [18]:
cosine_similarity(vectorized_symptoms_dict['anxiety'], get_vectorized_symptom(['i feel very anxious', 'i am scared in public', 'i have a strong feeling of anxiety'], model))
test_vector = get_vectorized_symptom([preprocess_description('i observed a large blood content in my sputum', wdl, ps)], model)
for key, value in vectorized_symptoms_dict.items():
    if cosine_similarity(test_vector, value) > 0.8:
        print(key)
        print(cosine_similarity(test_vector, value))

blood_in_sputum
0.8578588132710934


In [19]:
np.save('vsm_dict.npy', vectorized_symptoms_dict)